In [1]:
import ipsuite as ips

2023-04-06 16:09:21.167474: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 16:09:21.239776: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-06 16:09:21.854638: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/fzills/tools/cuda/cuda-11.8/lib64:
2023-04-06 16:09:21.854685: W tensorflow/compiler/xla/str

2023-04-06 16:09:23,011 (DEBUG): Welcome to IPS - the Interatomic Potential Suite!


In [2]:
mapping = ips.geometry.BarycenterMapping(data=None)

checks = [
    ips.analysis.ConnectivityCheck(),
    ips.analysis.NaNCheck()
]

with ips.Project() as project:
    data = ips.AddData(file="etoh_meoh_dimer.traj")

    test_data = ips.configuration_selection.RandomSelection(data=data, n_configurations=1000, name="test_data")
    train_data = ips.configuration_selection.RandomSelection(
        data=data, n_configurations=500, name="train_data",
        exclude_configurations=test_data.selected_configurations
    )
    val_data = ips.configuration_selection.RandomSelection(
        data=data, n_configurations=100, name="val_data",
        exclude_configurations=[test_data.selected_configurations, train_data.selected_configurations]
    )
    
    model = ips.models.MACE(data=train_data, test_data=val_data, max_num_epochs=250)

    predictions = ips.analysis.PredictWithModel(model=model, data=test_data)
    analysis = ips.analysis.AnalysePrediction(data=predictions)
    inter_intra_forces = ips.analysis.InterIntraForces(data=predictions)

    etoh = ips.configuration_generation.SmilesToAtoms(smiles="CCO", name="EtOH")
    meoh = ips.configuration_generation.SmilesToAtoms(smiles="CO", name="MeOH")
    packmol = ips.configuration_generation.Packmol(data=[etoh.atoms, meoh.atoms], count=[10, 10], box=12)
    geopt = ips.calculators.ASEGeoOpt(calculator=model.calc, data=packmol.atoms, optimizer="FIRE")
    md = ips.calculators.ASEMD(data=geopt.atoms, model=model, friction=0.01, sampling_rate=1, steps=5000, temperature=250, time_step=0.5)

    ips.analysis.BoxScaleAnalysis(data=md, model=model, mapping=mapping)

    # MD Stability
    stability_confs = ips.configuration_selection.RandomSelection(data=md, n_configurations=20, name="stability_confs")
    ips.analysis.MDStabilityAnalysis(
        data=stability_confs, model=model, checks=checks, max_steps=1000, initial_temperature=200, bins=10,
    )

project.run(repro=False)

Running DVC command: 'stage add --name AddData --force ...'
Running DVC command: 'stage add --name test_data --force ...'
Running DVC command: 'stage add --name train_data --force ...'
Running DVC command: 'stage add --name val_data --force ...'
Running DVC command: 'stage add --name MLModel --force ...'
Running DVC command: 'stage add --name PredictWithModel --force ...'
Running DVC command: 'stage add --name AnalysePrediction --force ...'
Running DVC command: 'stage add --name InterIntraForces --force ...'
Running DVC command: 'stage add --name EtOH --force ...'
Running DVC command: 'stage add --name MeOH --force ...'
Running DVC command: 'stage add --name Packmol --force ...'
Running DVC command: 'stage add --name ASEGeoOpt --force ...'
Running DVC command: 'stage add --name ASEMD --force ...'
Running DVC command: 'stage add --name BoxScaleAnalysis --force ...'
Running DVC command: 'stage add --name BoxScaleAnalysis_mapping --force ...'
Could not create .gitignore entry in /ssd/fzil

[![](https://mermaid.ink/img/pako:eNp9VFtr2zAU_itFz02JnWRx8jDYcNYOalrIoLB5GE06qbXakpGVdqH0v_cosTOmy_xgrO8iHX061ithigNZk12jXlhNtbn4lpfyYngkksmPknzabq5B3XWmJD__ZdMTW-QeM7MM5zk11OPmlpO0OfRwr4ELZoSSnmqBqs_qz5bRBk5q0XuiDwFR1dKuE_LREy9RvDF3Nx6RIfFVGtD40vSL0gz8pVYoKvKtob9EI8whWlIyDQsrVgN76qup70j-70h8hw2-uC3ws_FJm30BgX0mNvl7yp5aFbDZwIfzeBCmjkxuEwej6qoFfHHRgr7C0H5f8Wfmq23k_biviim5CwRm4zfQm4qHuiWxweMKQob51Ob9TJuIezL5eFQ5pgFehOFk6XTziGcRfBXG06nTruFll-M0c7eVBmLlNkwEP9cfwdMIvojg0fldwyy2g3mkosWAzyN45rbdgM_cBotUmsUqzSJHdibcM0tWoyONEK4jnZ4d5JLgD9JSwfGWfbUy7OUaWijJGj857Oi-wXu1lG8o3XfYw7DhwihN1kbv4ZLQvVHbg2Tj-KTJBX3UtCXrHW16RDsqvyv1dwzHOYrT7X685N_eAY3KuT0?type=png)](https://mermaid.live/edit#pako:eNp9VFtr2zAU_itFz02JnWRx8jDYcNYOalrIoLB5GE06qbXakpGVdqH0v_cosTOmy_xgrO8iHX061ithigNZk12jXlhNtbn4lpfyYngkksmPknzabq5B3XWmJD__ZdMTW-QeM7MM5zk11OPmlpO0OfRwr4ELZoSSnmqBqs_qz5bRBk5q0XuiDwFR1dKuE_LREy9RvDF3Nx6RIfFVGtD40vSL0gz8pVYoKvKtob9EI8whWlIyDQsrVgN76qup70j-70h8hw2-uC3ws_FJm30BgX0mNvl7yp5aFbDZwIfzeBCmjkxuEwej6qoFfHHRgr7C0H5f8Wfmq23k_biviim5CwRm4zfQm4qHuiWxweMKQob51Ob9TJuIezL5eFQ5pgFehOFk6XTziGcRfBXG06nTruFll-M0c7eVBmLlNkwEP9cfwdMIvojg0fldwyy2g3mkosWAzyN45rbdgM_cBotUmsUqzSJHdibcM0tWoyONEK4jnZ4d5JLgD9JSwfGWfbUy7OUaWijJGj857Oi-wXu1lG8o3XfYw7DhwihN1kbv4ZLQvVHbg2Tj-KTJBX3UtCXrHW16RDsqvyv1dwzHOYrT7X685N_eAY3KuT0)